In [ ]:
!pip install torchvision torch pytorch-grad-cam


ERROR: Could not find a version that satisfies the requirement pytorch-grad-cam (from versions: none)
ERROR: No matching distribution found for pytorch-grad-cam


In [ ]:
!pip install grad-cam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 60.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for grad-cam: filename=grad_cam-1.5.3-py3-none-any.whl size=38657 sha256=b4a8a102eb79371f7548fc29b5ffc090a33fdc4da43fd228a3bd07aeacc354f6
  Stored in directory: /root/.cache/pip/wheels/2e/ce/70/fe64f851895eae830b3c63ec7fc464cfa7c81aeb7ad4f68063
Successfully built grad-cam


In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
img_folder = '/content/drive/MyDrive/pocs/'
infected_folder = img_folder + 'infected/'
non_infected_folder = img_folder + 'notinfected/'

In [ ]:
import os
img_paths = []
labels = []

In [ ]:
# 2 folders 'infected' and 'non_infected'
for img_name in os.listdir(infected_folder):
    img_paths.append(os.path.join(infected_folder, img_name))
    labels.append(1)  # Label 1 for infected

In [ ]:
# non-infected images
for img_name in os.listdir(non_infected_folder):
    img_paths.append(os.path.join(non_infected_folder, img_name))
    labels.append(0)  # Label 0 for non-infected

In [ ]:
#PCOS class
class MedicalDataset(Dataset):
    def __init__(self, img_paths, labels, transform=None):
        self.img_paths = img_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB') #image is loaded as RGB
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [ ]:

dataset = MedicalDataset(img_paths, labels, transform=transform)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)


In [ ]:

# DenseNet201 model
model = models.densenet201(pretrained=True)
model.classifier = nn.Linear(1920, 2)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet201_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet201_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet201-c1103571.pth" to /root/.cache/torch/hub/checkpoints/densenet201-c1103571.pth
100%|██████████| 77.4M/77.4M [00:00<00:00, 92.0MB/s]


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 25

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in data_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(data_loader)}')


Epoch [1/25], Loss: 0.008696798825425022
Epoch [2/25], Loss: 3.6285243802260744e-05
Epoch [3/25], Loss: 1.8857313245296776e-05
Epoch [4/25], Loss: 9.7402877269236e-06
Epoch [5/25], Loss: 1.2410838213921027e-05
Epoch [6/25], Loss: 3.1317088961026275e-06
Epoch [7/25], Loss: 3.552772355302175e-06
Epoch [8/25], Loss: 5.892983187958041e-06
Epoch [9/25], Loss: 2.7944125675234814e-06
Epoch [10/25], Loss: 3.995269817650755e-06
Epoch [11/25], Loss: 0.048552717904629464
Epoch [12/25], Loss: 0.07232444973024106
Epoch [13/25], Loss: 0.004662744542528802
Epoch [14/25], Loss: 0.0004594754402307993
Epoch [15/25], Loss: 0.00011014844658897654
Epoch [16/25], Loss: 0.0028346094190767925
Epoch [17/25], Loss: 0.006244362644497508
Epoch [18/25], Loss: 0.0010247693672824447
Epoch [19/25], Loss: 6.207527922553186e-05
Epoch [20/25], Loss: 1.8780496658486296e-05
Epoch [21/25], Loss: 0.0034570761333011907
Epoch [22/25], Loss: 0.021756884249404442
Epoch [23/25], Loss: 0.0005818093151680112
Epoch [24/25], Loss: 0

In [ ]:

target_layers = [model.features.denseblock4]
cam = GradCAM(model=model, target_layers=target_layers)

In [ ]:
image_path = '/content/drive/MyDrive/pocs/infected/1 (8).jpg'
img = Image.open(image_path).convert('RGB')
input_tensor = transform(img).unsqueeze(0).to(device)



In [ ]:
targets = [ClassifierOutputTarget(1)]
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)


In [ ]:
img = img.resize((224, 224))
img_np = np.array(img)
if len(img_np.shape) == 2:
    img_np = cv2.cvtColor(img_np, cv2.COLOR_GRAY2BGR)

img_np = np.uint8(255 * img_np / np.max(img_np))

if len(grayscale_cam.shape) == 3:
    grayscale_cam = grayscale_cam[:, :, 0]
grayscale_cam = np.uint8(255 * grayscale_cam)
cam_image = show_cam_on_image(img_np, grayscale_cam, use_rgb=True)

In [ ]:
plt.imshow(cam_image)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)
plt.subplot(1, 2, 2)
plt.title('Grad-CAM')
plt.imshow(cam_image)
plt.show()

#Applying on diffrent data sample _ Infected

In [ ]:
image_path = '/content/drive/MyDrive/pocs/infected/1_(13).jpg'
img = Image.open(image_path).convert('RGB')
input_tensor = transform(img).unsqueeze(0).to(device)

In [ ]:
targets = [ClassifierOutputTarget(1)]
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)
img = img.resize((224, 224))
img_np = np.array(img) / 255.0
if len(grayscale_cam.shape) == 2:
    grayscale_cam = grayscale_cam[..., np.newaxis]

In [ ]:
cam_image = show_cam_on_image(img_np, grayscale_cam[0, :], use_rgb=True)

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)

plt.subplot(1, 2, 2)
plt.title('Grad-CAM')
plt.imshow(cam_image)

plt.show()

In [ ]:
image_path = '/content/drive/MyDrive/pocs/infected/1_(28).jpg'
img = Image.open(image_path).convert('RGB')
input_tensor = transform(img).unsqueeze(0).to(device)

targets = [ClassifierOutputTarget(1)]
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

img = img.resize((224, 224))
img_np = np.array(img) / 255.0

if len(grayscale_cam.shape) == 2:
    grayscale_cam = grayscale_cam[..., np.newaxis]

cam_image = show_cam_on_image(img_np, grayscale_cam[0, :], use_rgb=True)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)
plt.subplot(1, 2, 2)
plt.title('Grad-CAM')
plt.imshow(cam_image)

plt.show()

#ScoreCAM


In [ ]:
!pip install pytorch-gradcam

from pytorch_grad_cam import ScoreCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

image_path = '/content/drive/MyDrive/pocs/infected/1_(28).jpg'
img = Image.open(image_path).convert('RGB')
input_tensor = transform(img).unsqueeze(0).to(device)
cam = ScoreCAM(model=model, target_layers=target_layers)

targets = [ClassifierOutputTarget(1)]
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)
img = img.resize((224, 224))
img_np = np.array(img) / 255.0
if len(grayscale_cam.shape) == 2:
    grayscale_cam = grayscale_cam[..., np.newaxis]

cam_image = show_cam_on_image(img_np, grayscale_cam[0, :], use_rgb=True)
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)
plt.subplot(1, 2, 2)
plt.title('Score-CAM')
plt.imshow(cam_image)
plt.show()

#GradCAMPlusPlus

In [ ]:
!pip install pytorch-gradcam

from pytorch_grad_cam import GradCAMPlusPlus
from pytorch_grad_cam.utils.image import show_cam_on_image
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

image_path = '/content/drive/MyDrive/pocs/infected/1_(28).jpg'
img = Image.open(image_path).convert('RGB')
input_tensor = transform(img).unsqueeze(0).to(device)
cam = GradCAMPlusPlus(model=model, target_layers=target_layers)

targets = [ClassifierOutputTarget(1)]
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)
plt.subplot(1, 2, 2)
plt.title('Gradcam++')
plt.imshow(cam_image)
plt.show()

#XGradCAM

In [ ]:
from pytorch_grad_cam import XGradCAM
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget

cam = XGradCAM(model=model, target_layers=target_layers)
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)
cam_image = show_cam_on_image(img_np, grayscale_cam[0], use_rgb=True)
plt.imshow(cam_image)
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)
plt.subplot(1, 2, 2)
plt.title('XGradCAM')
plt.imshow(cam_image)
plt.show()


#EigenCAM

In [ ]:
from pytorch_grad_cam import EigenCAM

cam = EigenCAM(model=model, target_layers=target_layers)
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)
cam_image = show_cam_on_image(img_np, grayscale_cam[0], use_rgb=True)
plt.imshow(cam_image)
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)
plt.subplot(1, 2, 2)
plt.title('EigenCAM')
plt.imshow(cam_image)
plt.show()


#LayerCAM

In [ ]:
from pytorch_grad_cam import LayerCAM

cam = LayerCAM(model=model, target_layers=target_layers)
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)
cam_image = show_cam_on_image(img_np, grayscale_cam[0], use_rgb=True)
plt.imshow(cam_image)
plt.show()

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)
plt.subplot(1, 2, 2)
plt.title('LayerCAM')
plt.imshow(cam_image)
plt.show()

#Full-Gradient /  FullGrad

In [ ]:
from pytorch_grad_cam import FullGrad

cam = FullGrad(model=model, target_layers=target_layers)
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)
cam_image = show_cam_on_image(img_np, grayscale_cam[0], use_rgb=True)
plt.imshow(cam_image)
plt.show()


plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)
plt.subplot(1, 2, 2)
plt.title('Full-Gradient')
plt.imshow(cam_image)
plt.show()

#Thresholded CAM (Binary Mask)


In [ ]:
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)
threshold = 0.5
binary_cam = (grayscale_cam[0] > threshold).astype(float)
cam_image = show_cam_on_image(img_np, binary_cam, use_rgb=True)
plt.imshow(cam_image)
plt.show()
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)
plt.subplot(1, 2, 2)
plt.title('Thresholded CAM ')
plt.imshow(cam_image)
plt.show()


In [ ]:
from lime import lime_image
from skimage.segmentation import mark_boundaries
import torch
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

image_path = '/content/drive/MyDrive/pocs/infected/1_(13).jpg'
img = Image.open(image_path).convert('RGB')
img = img.resize((224, 224))
img_np = np.array(img)
input_tensor = transform(img).unsqueeze(0).to(device)

def predict_fn(images):
    inputs = [transform(Image.fromarray(image)).unsqueeze(0).to(device) for image in images]
    inputs = torch.cat(inputs, dim=0)
    with torch.no_grad():
        outputs = model(inputs)
    return outputs.cpu().numpy()

explainer = lime_image.LimeImageExplainer()
explanation = explainer.explain_instance(img_np,
                                         classifier_fn=predict_fn,
                                         top_labels=2,
                                         hide_color=0,
                                         num_samples=1000)

temp, mask = explanation.get_image_and_mask(label=1, positive_only=False, num_features=10, hide_rest=False)

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)

plt.subplot(1, 2, 2)
plt.title('LIME Explanation (Infected)')
plt.imshow(mark_boundaries(temp, mask))

plt.show()


OcclusionSensitivity

#2. SHAP (Shapley Additive exPlanations)

In [ ]:
import shap
import numpy as np
import torch
from PIL import Image

image_path = '/content/drive/MyDrive/pocs/infected/1_(13).jpg'
img = Image.open(image_path).convert('RGB')
img = img.resize((224, 224))
img_np = np.array(img) / 255.0
input_tensor = transform(img).unsqueeze(0).to(device)
explainer = shap.DeepExplainer(model, input_tensor)
shap_values = explainer.shap_values(input_tensor)
shap.image_plot(shap_values, np.array([img_np]))


In [ ]:
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

image_path = '/content/drive/MyDrive/pocs/infected/1 (13).jpg'
img = Image.open(image_path).convert('RGB')
img = img.resize((224, 224))
img_np = np.array(img) / 255.0
input_tensor = transform(img).unsqueeze(0).to(device)

def apply_occlusion(image, size=32, stride=8):
    heatmap = np.zeros((image.shape[1], image.shape[2]))
    for y in range(0, image.shape[1] - size + 1, stride):
        for x in range(0, image.shape[2] - size + 1, stride):
            occluded_image = image.clone()
            occluded_image[:, :, y:y+size, x:x+size] = 0
            output = model(occluded_image).squeeze()
            heatmap[y:y+size, x:x+size] = output[1].item()
    return heatmap

occlusion_heatmap = apply_occlusion(input_tensor)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)

plt.subplot(1, 2, 2)
plt.title('Occlusion Sensitivity')
plt.imshow(occlusion_heatmap, cmap='jet')
plt.show()


In [ ]:
import torch
from captum.attr import IntegratedGradients
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

image_path = '/content/drive/MyDrive/pocs/infected/1_(13).jpg'
img = Image.open(image_path).convert('RGB')
img = img.resize((224, 224))
img_np = np.array(img) / 255.0
input_tensor = transform(img).unsqueeze(0).to(device)

ig = IntegratedGradients(model)
attributions = ig.attribute(input_tensor, target=1)
attributions = attributions.squeeze().cpu().detach().numpy()

attributions = np.sum(np.abs(attributions), axis=0)
attributions = (attributions - np.min(attributions)) / (np.max(attributions) - np.min(attributions))

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)

plt.subplot(1, 2, 2)
plt.title('Integrated Gradients')
plt.imshow(attributions, cmap='hot')
plt.show()


In [ ]:
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

def add_noise(image, noise_level=0.1):
    noise = noise_level * torch.randn_like(image).to(device)
    return image + noise

def smoothgrad(input_tensor, n_samples=50, noise_level=0.1):
    smoothed_grad = torch.zeros_like(input_tensor)
    for i in range(n_samples):
        noisy_input = add_noise(input_tensor, noise_level)
        noisy_input.requires_grad = True
        output = model(noisy_input).squeeze()[1]
        output.backward()
        smoothed_grad += noisy_input.grad.abs()
    return smoothed_grad / n_samples

image_path = '/content/drive/MyDrive/pocs/infected/1_(13).jpg'
img = Image.open(image_path).convert('RGB')
img = img.resize((224, 224))
img_np = np.array(img) / 255.0
input_tensor = transform(img).unsqueeze(0).to(device)
smoothed_grad = smoothgrad(input_tensor).squeeze().cpu().detach().numpy()
smoothed_grad = np.sum(np.abs(smoothed_grad), axis=0)
smoothed_grad = (smoothed_grad - np.min(smoothed_grad)) / (np.max(smoothed_grad) - np.min(smoothed_grad))

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)

plt.subplot(1, 2, 2)
plt.title('SmoothGrad')
plt.imshow(smoothed_grad, cmap='hot')
plt.show()


In [ ]:
from captum.attr import GuidedBackprop
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

image_path = '/content/drive/MyDrive/pocs/infected/1 (13).jpg'
img = Image.open(image_path).convert('RGB')
img = img.resize((224, 224))
img_np = np.array(img) / 255.0
input_tensor = transform(img).unsqueeze(0).to(device)
guided_bp = GuidedBackprop(model)
attributions = guided_bp.attribute(input_tensor, target=1)
attributions = attributions.squeeze().cpu().detach().numpy()
attributions = np.sum(attributions, axis=0)
attributions = np.maximum(attributions, 0)
attributions = (attributions - attributions.min()) / (attributions.max() - attributions.min())
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)
plt.subplot(1, 2, 2)
plt.title('Guided Backpropagation')
plt.imshow(attributions, cmap='hot')
plt.show()


In [ ]:
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

image_path = '/content/drive/MyDrive/pocs/infected/1 (13).jpg'
img = Image.open(image_path).convert('RGB')
img = img.resize((224, 224))
img_np = np.array(img) / 255.0
input_tensor = transform(img).unsqueeze(0).to(device)

with torch.no_grad():
    model.eval()
    output = model(input_tensor)
    output_score = output[0, 1].item()

def score_features(input_tensor, score_function):
    scores = np.zeros(input_tensor.shape[-2:])
    for y in range(input_tensor.shape[-2]):
        for x in range(input_tensor.shape[-1]):
            perturbed_input = input_tensor.clone()
            perturbed_input[:, :, y, x] = 0
            perturbed_output = model(perturbed_input)[0, 1].item()
            scores[y, x] = output_score - perturbed_output
    return scores

scores = score_features(input_tensor, output_score)
scores = (scores - scores.min()) / (scores.max() - scores.min())

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)
plt.subplot(1, 2, 2)
plt.title('SBE Heatmap')
plt.imshow(scores, cmap='hot')
plt.show()


In [ ]:
import torch
import numpy as np
from PIL import Image, ImageFilter
import matplotlib.pyplot as plt

image_path = '/content/drive/MyDrive/pocs/infected/1_(13).jpg'
img = Image.open(image_path).convert('RGB')
img = img.resize((224, 224))
img_np = np.array(img) / 255.0
input_tensor = transform(img).unsqueeze(0).to(device)

def apply_perturbation(img_np, patch_size=32, stride=8):
    perturbed_img = img.copy()
    for y in range(0, img_np.shape[0] - patch_size + 1, stride):
        for x in range(0, img_np.shape[1] - patch_size + 1, stride):
            patch = img.crop((x, y, x + patch_size, y + patch_size))
            blurred_patch = patch.filter(ImageFilter.GaussianBlur(radius=5))
            perturbed_img.paste(blurred_patch, (x, y))
    return np.array(perturbed_img) / 255.0

perturbed_img_np = apply_perturbation(img_np)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(img_np)

plt.subplot(1, 2, 2)
plt.title('Perturbation-Based Saliency Map')
plt.imshow(perturbed_img_np)
plt.show()
